In [3]:
import pandas as pd
df_source = pd.read_csv('../data/final_test/final/reexperiment/combined_sourcedf_final_rebuild.csv')

In [15]:
if 'ewca_civ_2018_162#para_13' in df_source.para_id.unique():
    print("Yes")
else:
    print("No")


Yes


In [4]:
source_pairs = set(zip(df_source['para_id'], df_source['section_id']))


In [5]:
df_openai_new = pd.read_csv('../data/final_test/final/reexperiment/fewhot/11August/redo-gpt-4o-mini_output-combined.csv')

In [ ]:
df_openai_new.para_id

9001

In [11]:
df_openai_new.para_id.unique()

array(['eat_2022_192#para_12', 'eat_2022_192#para_18',
       'eat_2022_192#para_21', ..., 'ewfc_b_2024_69#para_203',
       'uksc_2013_11#para_42', 'ewhc_qb_2017_294#para_29'],
      shape=(9002,), dtype=object)

In [16]:
if 'ewca_civ_2018_162#para_13' in df_openai_new.para_id.unique():
    print("Yes")

Yes


In [30]:
sample

,custom_id,para_id,section_id,phrase_index,case_law_excerpt,legislation_excerpt,confidence,reasoning,para_text,section_text,line_number,status
4858,request_2477,ewca_civ_2018_162#para_13,id/ukpga/2002/29_section-170,0,The benefits that the Appellant obtained throu...,the defendant has a criminal lifestyle but has...,High,The case law reflects upon the irrelevance of ...,13. We refused permission to adduce this evide...,170 No order made: reconsideration of benefit ...,2431,SUCCESS
4859,request_2477,ewca_civ_2018_162#para_13,id/ukpga/2002/29_section-170,0,The benefits that the Appellant obtained throu...,the defendant has a criminal lifestyle but has...,High,The case law reflects upon the irrelevance of ...,13. We refused permission to adduce this evide...,170 No order made: reconsideration of benefit ...,2431,SUCCESS
4860,request_2478,ewca_civ_2018_162#para_13,id/ukpga/2002/29_section-20,0,"Moreover, the argument to be made on the stren...",may make a confiscation order under that section,High,The case law discusses the implications of con...,13. We refused permission to adduce this evide...,20 No order made: reconsideration of benefit (...,2432,SUCCESS
4861,request_2478,ewca_civ_2018_162#para_13,id/ukpga/2002/29_section-20,0,"Moreover, the argument to be made on the stren...",may make a confiscation order under that section,High,The case law discusses the implications of con...,13. We refused permission to adduce this evide...,20 No order made: reconsideration of benefit (...,2432,SUCCESS


In [24]:
sample = df_openai_new[df_openai_new.para_id == 'ewca_civ_2018_162#para_13']

In [34]:
sample['case_law_excerpt'].values[0]

'The benefits that the Appellant obtained through his criminal tipping on the Land are irrelevant to that claim or its quantification. Moreover, the argument to be made on the strength of this evidence would appear to involve a claim to share in the Appellant’s proceeds of crime, which might well be open to objection in principle.'

In [33]:
sample['legislation_excerpt'].values[3]

'may make a confiscation order under that section'

In [31]:
sample['reasoning'].values[3]

"The case law discusses the implications of considering the benefits gained through unlawful actions, which relates directly to the court's ability to make confiscation orders as outlined in the legislation."

In [5]:
df_source.columns

Index(['case_uri', 'para_id', 'paragraphs', 'references', 'if_law_applied',
       'application_of_law_phrases', 'reason', 'if_law_applied_llama',
       'application_of_law_phrases_llama', 'reason_llama',
       'if_law_applied_claude', 'application_of_law_phrases_claude',
       'reason_claude', 'confidence', 'agreement_with', 'final_annotation',
       'case_name', 'section_id', 'section_text'],
      dtype='object')

In [6]:
df_openai_new.columns

Index(['custom_id', 'para_id', 'section_id', 'phrase_index',
       'case_law_excerpt', 'legislation_excerpt', 'confidence', 'reasoning',
       'para_text', 'section_text', 'line_number', 'status'],
      dtype='object')

In [7]:
df_openai_new = df_openai_new[df_openai_new.status == 'SUCCESS']
# Filter df_openai_new to only keep rows where (para_id, section_id) exists in source_pairs
df_openai_new = df_openai_new[df_openai_new.apply(lambda row: (row['para_id'], row['section_id']) in source_pairs, axis=1)]

print(f"Length of filtered df_openai_new: {len(df_openai_new)}")

# Get existing pairs in df_openai_new
existing_pairs = set(zip(df_openai_new['para_id'], df_openai_new['section_id']))

# Find missing pairs that are in source_pairs but not in df_openai_new
missing_pairs = source_pairs - existing_pairs

# Create records for missing pairs
missing_records = []
for para_id, section_id in missing_pairs:
    # Get the corresponding row from df_source
    source_row = df_source[(df_source['para_id'] == para_id) & 
                          (df_source['section_id'] == section_id)].iloc[0]
    
    missing_record = {
        'custom_id': '000',
        'para_id': para_id,
        'section_id': section_id,
        'phrase_index': -1,
        'case_law_excerpt': None,
        'legislation_excerpt': None,
        'confidence': 'Low',
        'reasoning': None,
        'para_text': source_row['paragraphs'],  # assuming this column exists in df_source
        'section_text': source_row['section_text'],  # assuming this column exists in df_source
        'line_number': '000',
        'status': 'Dummy',
        'is_valid': False  # assuming missing records are not valid
    }
    missing_records.append(missing_record)

# Convert to DataFrame and concatenate
if missing_records:
    df_missing = pd.DataFrame(missing_records)
    df_openai_new = pd.concat([df_openai_new, df_missing], ignore_index=True)

print(f"Length after adding missing records: {len(df_openai_new)}")
print(f"Added {len(missing_records)} missing records")

Length of filtered df_openai_new: 35640
Length after adding missing records: 35824
Added 184 missing records


In [8]:
df_openai_new.para_id.nunique()

9002

In [9]:
df_openai_new.confidence.value_counts()

confidence
High    30044
Low      5780
Name: count, dtype: int64

In [ ]:
import re
def remove_special_chars(s):
    if not isinstance(s, str):
        return s
    # Remove all non-alphanumeric and non-space characters
    return re.sub(r'[^A-Za-z0-9 ]+', '', s)
def check_valid(row):
    #case_ok = str(row.get('case_law_excerpt', '')) in str(row.get('para_text', ''))
    legis_ok = str(row.get('legislation_excerpt', '')) in str(row.get('section_text', ''))
    if legis_ok:#if case_ok and legis_ok:
        return True
    cleaned_caselaw_excerpt = remove_special_chars(row.get('case_law_excerpt', ''))
    cleaned_legislation_excerpt = remove_special_chars(row.get('legislation_excerpt', ''))
    cleaned_para_text = remove_special_chars(row.get('para_text', ''))
    cleaned_section_text = remove_special_chars(row.get('section_text', ''))
    case_ok = str(cleaned_caselaw_excerpt) in str(cleaned_para_text)
    legis_ok = str(cleaned_legislation_excerpt) in str(cleaned_section_text)    
    #return case_ok and legis_ok
    return legis_ok

In [12]:
df_openai_new['is_valid'] = df_openai_new.apply(check_valid, axis=1)

In [13]:
# Set confidence to 'Low' for records where is_valid is False
df_openai_new.loc[df_openai_new['is_valid'] == False, 'confidence'] = 'Low'

# Check the result
print("Confidence distribution after update:")
print(df_openai_new['confidence'].value_counts())


Confidence distribution after update:
confidence
High    23248
Low     12576
Name: count, dtype: int64


In [14]:
High_ids_openai = df_openai_new[df_openai_new['confidence'] == 'High']['para_id'].nunique()
print(f"Number of para_ids with at least one High entry: {High_ids_openai}")

Number of para_ids with at least one High entry: 7701


In [15]:

# Get para_ids that have at least one High confidence record
para_ids_with_high = df_openai_new[df_openai_new['confidence'] == 'High']['para_id'].unique()

# Get para_ids that have NO High confidence records (only Low)
para_ids_only_low = df_openai_new[~df_openai_new['para_id'].isin(para_ids_with_high)]['para_id'].unique()

# Filter the dataframe:
# 1. Keep High confidence records for para_ids that have at least one High
# 2. Keep Low confidence records only for para_ids that have NO High records
df_openai_filtered = df_openai_new[
    ((df_openai_new['confidence'] == 'High') & (df_openai_new['para_id'].isin(para_ids_with_high))) |
    ((df_openai_new['confidence'] == 'Low') & (df_openai_new['para_id'].isin(para_ids_only_low)))
]

print(f"\nAfter filtering:")
print(f"Total records: {len(df_openai_filtered)}")
print(f"High confidence records: {len(df_openai_filtered[df_openai_filtered['confidence'] == 'High'])}")
print(f"Low confidence records: {len(df_openai_filtered[df_openai_filtered['confidence'] == 'Low'])}")
print(f"Unique para_ids: {df_openai_filtered['para_id'].nunique()}")

print(f"\nConfidence distribution:")
print(df_openai_filtered['confidence'].value_counts())


After filtering:
Total records: 28382
High confidence records: 23248
Low confidence records: 5134
Unique para_ids: 9002

Confidence distribution:
confidence
High    23248
Low      5134
Name: count, dtype: int64


In [16]:
High_ids_openai = df_openai_filtered[df_openai_filtered['confidence'] == 'High']['para_id'].nunique()
print(f"Number of para_ids with at least one High entry: {High_ids_openai}")

Low_ids_openai = df_openai_filtered[df_openai_filtered['confidence'] == 'Low']['para_id'].nunique()
print(f"Number of para_ids with at least one Low entry: {Low_ids_openai}")

Number of para_ids with at least one High entry: 7701
Number of para_ids with at least one Low entry: 1301


In [18]:
import pandas as pd

df_llama = pd.read_csv('../data/final_test/final/reexperiment/fewhot/11August/llama_combined_output_final.csv')

In [18]:
df_llama = df_llama[df_llama.status == 'SUCCESS']

In [19]:
df_llama = df_llama[df_llama.apply(lambda row: (row['para_id'], row['section_id']) in source_pairs, axis=1)]

print(f"Length of filtered df_llama: {len(df_llama)}")

# Get existing pairs in df_llama
existing_pairs_llama = set(zip(df_llama['para_id'], df_llama['section_id']))

# Find missing pairs that are in source_pairs but not in df_llama
missing_pairs_llama = source_pairs - existing_pairs_llama

# Create records for missing pairs
missing_records = []
for para_id, section_id in missing_pairs_llama:
    # Get the corresponding row from df_source
    source_row = df_source[(df_source['para_id'] == para_id) & 
                          (df_source['section_id'] == section_id)].iloc[0]
    
    missing_record = {
        'custom_id': '000',
        'para_id': para_id,
        'section_id': section_id,
        'phrase_index': -1,
        'case_law_excerpt': None,
        'legislation_excerpt': None,
        'confidence': 'Low',
        'reasoning': None,
        'para_text': source_row['paragraphs'],  # assuming this column exists in df_source
        'section_text': source_row['section_text'],  # assuming this column exists in df_source
        'line_number': '000',
        'status': 'Dummy',
        'is_valid': False  # assuming missing records are not valid
    }
    missing_records.append(missing_record)

# Convert to DataFrame and concatenate
if missing_records:
    df_missing = pd.DataFrame(missing_records)
    df_llama = pd.concat([df_llama, df_missing], ignore_index=True)

print(f"Length after adding missing records: {len(df_llama)}")
print(f"Added {len(missing_records)} missing records")

Length of filtered df_llama: 21245
Length after adding missing records: 22696
Added 1451 missing records


In [19]:
df_llama[df_llama.para_id == 'ewca_civ_2018_162#para_13']


,custom_id,para_id,section_id,phrase_index,case_law_excerpt,legislation_excerpt,confidence,reasoning,para_text,section_text,line_number,status
2098,request_2477,ewca_civ_2018_162#para_13,id/ukpga/2002/29_section-170,0,the benefits that the Appellant obtained throu...,evidence which was not available to the prosec...,Low,The court's decision to refuse permission to a...,13. We refused permission to adduce this evide...,170 No order made: reconsideration of benefit ...,2466,SUCCESS
2112,request_2478,ewca_civ_2018_162#para_13,id/ukpga/2002/29_section-20,0,the benefits that the Appellant obtained throu...,the court concludes that it would have decided...,Low,The court's decision to refuse permission to a...,13. We refused permission to adduce this evide...,20 No order made: reconsideration of benefit (...,2483,SUCCESS


In [20]:
df_llama['is_valid'] = df_llama.apply(check_valid, axis=1)

# Set confidence to 'Low' for records where is_valid is False
df_llama.loc[df_llama['is_valid'] == False, 'confidence'] = 'Low'

# Check the result
print("Confidence distribution after update:")
print(df_llama['confidence'].value_counts())

Confidence distribution after update:
confidence
Low       14957
High       7624
Medium      115
Name: count, dtype: int64


In [21]:
df_llama.loc[df_llama['confidence'] == 'Medium', 'confidence'] = 'High'



In [22]:
print(df_llama['confidence'].value_counts())

confidence
Low     14957
High     7739
Name: count, dtype: int64


In [23]:
High_ids_llama = df_llama[df_llama['confidence'] == 'High']['para_id'].nunique()
print(f"Number of para_ids with at least one High entry: {High_ids_llama}")

Number of para_ids with at least one High entry: 5492


In [24]:

# Get para_ids that have at least one High confidence record
para_ids_with_high_llama = df_llama[df_llama['confidence'] == 'High']['para_id'].unique()

# Get para_ids that have NO High confidence records (only Low)
para_ids_only_low_llama = df_llama[~df_llama['para_id'].isin(para_ids_with_high_llama)]['para_id'].unique()

# Filter the dataframe:
# 1. Keep High confidence records for para_ids that have at least one High
# 2. Keep Low confidence records only for para_ids that have NO High records
df_llama_filtered = df_llama[
    ((df_llama['confidence'] == 'High') & (df_llama['para_id'].isin(para_ids_with_high_llama))) |
    ((df_llama['confidence'] == 'Low') & (df_llama['para_id'].isin(para_ids_only_low_llama)))
]

print(f"\nAfter filtering:")
print(f"Total records: {len(df_llama_filtered)}")
print(f"High confidence records: {len(df_llama_filtered[df_llama_filtered['confidence'] == 'High'])}")
print(f"Low confidence records: {len(df_llama_filtered[df_llama_filtered['confidence'] == 'Low'])}")
print(f"Unique para_ids: {df_llama_filtered['para_id'].nunique()}")

print(f"\nConfidence distribution:")
print(df_llama_filtered['confidence'].value_counts())


After filtering:
Total records: 16816
High confidence records: 7739
Low confidence records: 9077
Unique para_ids: 9002

Confidence distribution:
confidence
Low     9077
High    7739
Name: count, dtype: int64


In [25]:
High_ids_llama = df_llama_filtered[df_llama_filtered['confidence'] == 'High']['para_id'].nunique()
print(f"Number of para_ids with at least one High entry: {High_ids_llama}")

Low_ids_llama = df_llama_filtered[df_llama_filtered['confidence'] == 'Low']['para_id'].nunique()
print(f"Number of para_ids with at least one Low entry: {Low_ids_llama}")



Number of para_ids with at least one High entry: 5492
Number of para_ids with at least one Low entry: 3510


In [20]:
df_deepseek = pd.read_csv('../data/final_test/final/reexperiment/fewhot/11August/deepseek_combined_output_final.csv')

In [23]:
'ewca_civ_2018_162#para_13' in df_deepseek.para_id.unique()
df_deepseek[df_deepseek['para_id'] == 'ewca_civ_2018_162#para_13']

,custom_id,para_id,section_id,phrase_index,case_law_excerpt,legislation_excerpt,confidence,reasoning,para_text,section_text,line_number,status
2198,request_2478,ewca_civ_2018_162#para_13,id/ukpga/2002/29_section-20,0,We refused permission to adduce this evidence....,If this section applies the court— (a) must ma...,High,The case law excerpt discusses the refusal to ...,13. We refused permission to adduce this evide...,20 No order made: reconsideration of benefit (...,2474,SUCCESS
2207,request_2477,ewca_civ_2018_162#para_13,id/ukpga/2002/29_section-170,0,The benefits that the Appellant obtained throu...,the court has decided that— (a) the defendant ...,High,The case law discusses the irrelevance of crim...,13. We refused permission to adduce this evide...,170 No order made: reconsideration of benefit ...,2485,SUCCESS


In [28]:
df_deepseek = df_deepseek[df_deepseek.status == 'SUCCESS']

In [29]:
# Filter df_deepseek to only keep rows where (para_id, section_id) exists in source_pairs
df_deepseek = df_deepseek[df_deepseek.apply(lambda row: (row['para_id'], row['section_id']) in source_pairs, axis=1)]

print(f"Length of filtered df_deepseek: {len(df_deepseek)}")

# Get existing pairs in df_deepseek
existing_pairs_deepseek = set(zip(df_deepseek['para_id'], df_deepseek['section_id']))

# Find missing pairs that are in source_pairs but not in df_deepseek
missing_pairs_deepseek = source_pairs - existing_pairs_deepseek

# Create records for missing pairs
missing_records_deepseek = []
for para_id, section_id in missing_pairs_deepseek:
    # Get the corresponding row from df_source
    source_row = df_source[(df_source['para_id'] == para_id) & 
                          (df_source['section_id'] == section_id)].iloc[0]
    
    missing_record_deepseek = {
        'custom_id': '000',
        'para_id': para_id,
        'section_id': section_id,
        'phrase_index': -1,
        'case_law_excerpt': None,
        'legislation_excerpt': None,
        'confidence': 'Low',
        'reasoning': None,
        'para_text': source_row['paragraphs'],  # assuming this column exists in df_source
        'section_text': source_row['section_text'],  # assuming this column exists in df_source
        'line_number': '000',
        'status': 'Dummy',
        'is_valid': False  # assuming missing records are not valid
    }
    missing_records_deepseek.append(missing_record_deepseek)

# Convert to DataFrame and concatenate
if missing_records_deepseek:
    df_missing_deepseek = pd.DataFrame(missing_records_deepseek)
    df_deepseek = pd.concat([df_deepseek, df_missing_deepseek], ignore_index=True)

print(f"Length after adding missing records: {len(df_deepseek)}")
print(f"Added {len(missing_records_deepseek)} missing records")

Length of filtered df_deepseek: 21792
Length after adding missing records: 22247
Added 455 missing records


In [30]:
df_deepseek['is_valid'] = df_deepseek.apply(check_valid, axis=1)

# Set confidence to 'Low' for records where is_valid is False
df_deepseek.loc[df_deepseek['is_valid'] == False, 'confidence'] = 'Low'

# Check the result
print("Confidence distribution after update:")
print(df_deepseek['confidence'].value_counts())

Confidence distribution after update:
confidence
Low         11413
High        10753
Medium         64
Moderate       16
No match        1
Name: count, dtype: int64


In [31]:
df_deepseek.loc[df_deepseek['confidence'] == 'Medium', 'confidence'] = 'High'
df_deepseek.loc[df_deepseek['confidence'] == 'Moderate', 'confidence'] = 'High'
df_deepseek.loc[df_deepseek['confidence'] == 'No match', 'confidence'] = 'Low'


In [32]:
print("Confidence distribution after update:")
print(df_deepseek['confidence'].value_counts())

Confidence distribution after update:
confidence
Low     11414
High    10833
Name: count, dtype: int64


In [33]:
High_ids_deepseek = df_deepseek[df_deepseek['confidence'] == 'High']['para_id'].nunique()
print(f"Number of para_ids with at least one High entry: {High_ids_deepseek}")

Number of para_ids with at least one High entry: 7137


In [34]:
# Get para_ids that have at least one High confidence record
para_ids_with_high_deepseek = df_deepseek[df_deepseek['confidence'] == 'High']['para_id'].unique()

# Get para_ids that have NO High confidence records (only Low)
para_ids_only_low_deepseek = df_deepseek[~df_deepseek['para_id'].isin(para_ids_with_high_deepseek)]['para_id'].unique()

# Filter the dataframe:
# 1. Keep High confidence records for para_ids that have at least one High
# 2. Keep Low confidence records only for para_ids that have NO High records
df_deepseek_filtered = df_deepseek[
    ((df_deepseek['confidence'] == 'High') & (df_deepseek['para_id'].isin(para_ids_with_high_deepseek))) |
    ((df_deepseek['confidence'] == 'Low') & (df_deepseek['para_id'].isin(para_ids_only_low_deepseek)))
]

print(f"\nAfter filtering:")
print(f"Total records: {len(df_deepseek_filtered)}")
print(f"High confidence records: {len(df_deepseek_filtered[df_deepseek_filtered['confidence'] == 'High'])}")
print(f"Low confidence records: {len(df_deepseek_filtered[df_deepseek_filtered['confidence'] == 'Low'])}")
print(f"Unique para_ids: {df_deepseek_filtered['para_id'].nunique()}")

print(f"\nConfidence distribution:")
print(df_deepseek_filtered['confidence'].value_counts())


After filtering:
Total records: 16065
High confidence records: 10833
Low confidence records: 5232
Unique para_ids: 9002

Confidence distribution:
confidence
High    10833
Low      5232
Name: count, dtype: int64


In [35]:
High_ids_deepseek = df_deepseek_filtered[df_deepseek_filtered['confidence'] == 'High']['para_id'].nunique()
print(f"Number of para_ids with at least one High entry: {High_ids_deepseek}")

Low_ids_deepseek = df_deepseek_filtered[df_deepseek_filtered['confidence'] == 'Low']['para_id'].nunique()
print(f"Number of para_ids with at least one Low entry: {Low_ids_deepseek}")

Number of para_ids with at least one High entry: 7137
Number of para_ids with at least one Low entry: 1865


In [41]:
len(set(df_deepseek_filtered[df_deepseek_filtered['confidence'] == 'High']['para_id'].unique())&set(df_llama_filtered[df_llama_filtered['confidence'] == 'High']['para_id'].unique())&set(df_openai_filtered[df_openai_filtered['confidence'] == 'High']['para_id'].unique()))

4631

In [38]:

# First, let's ensure each dataframe has only one record per para_id
# For para_ids with multiple Low records, keep only one (this should mainly affect df_deepseek_filtered)

def keep_one_record_per_para_id(df):
    """Keep one record per para_id - prefer High confidence, if all Low then keep first"""
    # Group by para_id and keep the first record (High confidence records should already be filtered appropriately)
    return df.groupby('para_id').first().reset_index()

df_deepseek_final = keep_one_record_per_para_id(df_deepseek_filtered)
df_llama_final = keep_one_record_per_para_id(df_llama_filtered)
df_openai_final = keep_one_record_per_para_id(df_openai_filtered)

print("Records per dataframe after keeping one per para_id:")
print(f"DeepSeek: {len(df_deepseek_final)}")
print(f"Llama: {len(df_llama_final)}")
print(f"OpenAI: {len(df_openai_final)}")

# Get all unique para_ids across all three dataframes
all_para_ids = set(df_deepseek_final['para_id'].unique()) | \
               set(df_llama_final['para_id'].unique()) | \
               set(df_openai_final['para_id'].unique())

X = len(all_para_ids)
print(f"\nTotal unique para_ids (X): {X}")

# Create a comprehensive dataframe with confidence for each model
results = []

for para_id in all_para_ids:
    # Get confidence for each model (None if para_id not present)
    deepseek_conf = df_deepseek_final[df_deepseek_final['para_id'] == para_id]['confidence'].iloc[0] if para_id in df_deepseek_final['para_id'].values else None
    llama_conf = df_llama_final[df_llama_final['para_id'] == para_id]['confidence'].iloc[0] if para_id in df_llama_final['para_id'].values else None
    openai_conf = df_openai_final[df_openai_final['para_id'] == para_id]['confidence'].iloc[0] if para_id in df_openai_final['para_id'].values else None
    
    # Count High and Low confidences
    confidences = [deepseek_conf, llama_conf, openai_conf]
    high_count = confidences.count('High')
    low_count = confidences.count('Low')
    
    results.append({
        'para_id': para_id,
        'deepseek_conf': deepseek_conf,
        'llama_conf': llama_conf,
        'openai_conf': openai_conf,
        'high_count': high_count,
        'low_count': low_count
    })

df_analysis = pd.DataFrame(results)

# Categorize based on High/Low distribution
def categorize_confidence(row):
    if row['high_count'] == 3:
        return 'a_all_high'
    elif row['high_count'] == 2 and row['low_count'] == 1:
        return 'b_2high_1low'
    elif row['high_count'] == 1 and row['low_count'] == 2:
        return 'c_1high_2low'
    elif row['low_count'] == 3:
        return 'd_all_low'
    else:
        return 'other'  # This shouldn't happen if all para_ids have 3 entries

df_analysis['category'] = df_analysis.apply(categorize_confidence, axis=1)

# Display results
print("\nDistribution of para_ids by confidence pattern:")
category_counts = df_analysis['category'].value_counts()
for category, count in category_counts.items():
    percentage = (count / X) * 100
    print(f"{category}: {count} para_ids ({percentage:.1f}%)")

print(f"\nVerification: {category_counts.sum()} = {X} ✓" if category_counts.sum() == X else f"\nError: {category_counts.sum()} ≠ {X}")


Records per dataframe after keeping one per para_id:
DeepSeek: 9002
Llama: 9002
OpenAI: 9002

Total unique para_ids (X): 9002

Distribution of para_ids by confidence pattern:
a_all_high: 4631 para_ids (51.4%)
b_2high_1low: 2604 para_ids (28.9%)
c_1high_2low: 1229 para_ids (13.7%)
d_all_low: 538 para_ids (6.0%)

Verification: 9002 = 9002 ✓


In [44]:
# Get the para_ids that are in the 'd_all_low' category
d_all_low_para_ids = df_analysis[df_analysis['category'] == 'd_all_low']['para_id'].tolist()

print(f"Number of para_ids with all low confidence: {len(d_all_low_para_ids)}")

# Filter df_source to keep only rows with para_ids in d_all_low
df_source_all_low = df_source[df_source['para_id'].isin(d_all_low_para_ids)].copy()

print(f"Number of records in df_source_all_low: {len(df_source_all_low)}")
print(f"Unique para_ids in df_source_all_low: {df_source_all_low['para_id'].nunique()}")

# Verify that we got all the d_all_low para_ids
unique_para_ids_in_result = df_source_all_low['para_id'].nunique()
print(f"Verification: Expected {len(d_all_low_para_ids)} para_ids, got {unique_para_ids_in_result}")

# Show some basic info about the filtered dataframe
print(f"\nDataFrame shape: {df_source_all_low.shape}")
print(f"Columns: {list(df_source_all_low.columns)}")

# Show first few rows
print(f"\nFirst 5 rows:")
print(df_source_all_low.head())

Number of para_ids with all low confidence: 538
Number of records in df_source_all_low: 1076
Unique para_ids in df_source_all_low: 538
Verification: Expected 538 para_ids, got 538

DataFrame shape: (1076, 19)
Columns: ['case_uri', 'para_id', 'paragraphs', 'references', 'if_law_applied', 'application_of_law_phrases', 'reason', 'if_law_applied_llama', 'application_of_law_phrases_llama', 'reason_llama', 'if_law_applied_claude', 'application_of_law_phrases_claude', 'reason_claude', 'confidence', 'agreement_with', 'final_annotation', 'case_name', 'section_id', 'section_text']

First 5 rows:
                                              case_uri               para_id  \
124  https://caselaw.nationalarchives.gov.uk/eat/20...  eat_2024_17#para_100   
125  https://caselaw.nationalarchives.gov.uk/eat/20...  eat_2024_17#para_100   
128  https://caselaw.nationalarchives.gov.uk/eat/20...  eat_2024_17#para_112   
129  https://caselaw.nationalarchives.gov.uk/eat/20...  eat_2024_17#para_112   
130  ht

In [56]:
# Create a wide format dataframe with one row per (para_id, section_id) and columns for each model
def create_wide_analysis_df():
    # Start with the analysis dataframe to get all para_ids and their categories
    df_wide = df_analysis[['para_id', 'category']].copy()
    
    # We need to get section_ids, so let's merge with one of the model dataframes first
    # Get all unique (para_id, section_id) combinations from all models
    all_combinations = []
    
    for df_model in [df_deepseek_final, df_llama_final, df_openai_final]:
        combinations = df_model[['para_id', 'section_id']].drop_duplicates()
        all_combinations.append(combinations)
    
    # Combine all combinations and remove duplicates
    all_combos = pd.concat(all_combinations).drop_duplicates().reset_index(drop=True)
    
    # Merge with category information
    df_wide = all_combos.merge(df_analysis[['para_id', 'category']], on='para_id', how='left')
    
    # Add columns from deepseek
    deepseek_cols = df_deepseek_final[['para_id', 'section_id', 'case_law_excerpt', 'legislation_excerpt', 'confidence', 'reasoning']].copy()
    deepseek_cols = deepseek_cols.rename(columns={
        'case_law_excerpt': 'case_law_excerpt_deepseek',
        'legislation_excerpt': 'legislation_excerpt_deepseek',
        'confidence': 'confidence_deepseek',
        'reasoning': 'reasoning_deepseek'
    })
    df_wide = df_wide.merge(deepseek_cols, on=['para_id', 'section_id'], how='left')
    
    # Add columns from llama
    llama_cols = df_llama_final[['para_id', 'section_id', 'case_law_excerpt', 'legislation_excerpt', 'confidence', 'reasoning']].copy()
    llama_cols = llama_cols.rename(columns={
        'case_law_excerpt': 'case_law_excerpt_llama',
        'legislation_excerpt': 'legislation_excerpt_llama',
        'confidence': 'confidence_llama',
        'reasoning': 'reasoning_llama'
    })
    df_wide = df_wide.merge(llama_cols, on=['para_id', 'section_id'], how='left')
    
    # Add columns from openai
    openai_cols = df_openai_final[['para_id', 'section_id', 'case_law_excerpt', 'legislation_excerpt', 'confidence', 'reasoning']].copy()
    openai_cols = openai_cols.rename(columns={
        'case_law_excerpt': 'case_law_excerpt_openai',
        'legislation_excerpt': 'legislation_excerpt_openai',
        'confidence': 'confidence_openai',
        'reasoning': 'reasoning_openai'
    })
    df_wide = df_wide.merge(openai_cols, on=['para_id', 'section_id'], how='left')
    
    # Add additional columns from df_source instead of model dataframes
    base_cols = df_source[['para_id', 'section_id', 'paragraphs', 'section_text']].drop_duplicates()
    df_wide = df_wide.merge(base_cols, on=['para_id', 'section_id'], how='left')
    
    return df_wide

# Create the wide format dataframe
df_final_wide = create_wide_analysis_df()

print(f"Shape of wide analysis dataframe: {df_final_wide.shape}")
print(f"Unique para_ids: {df_final_wide['para_id'].nunique()}")
print(f"Unique (para_id, section_id) combinations: {len(df_final_wide)}")

print(f"\nColumns in final wide dataframe:")
for col in df_final_wide.columns:
    print(f"  {col}")

print(f"\nDistribution by category:")
print(df_final_wide['category'].value_counts())

print(f"\nFirst 3 rows (showing key columns):")
key_cols = ['para_id', 'section_id', 'category', 'confidence_deepseek', 'confidence_llama', 'confidence_openai']
print(df_final_wide[key_cols].head(3))

Shape of wide analysis dataframe: (14724, 17)
Unique para_ids: 9002
Unique (para_id, section_id) combinations: 14724

Columns in final wide dataframe:
  para_id
  section_id
  category
  case_law_excerpt_deepseek
  legislation_excerpt_deepseek
  confidence_deepseek
  reasoning_deepseek
  case_law_excerpt_llama
  legislation_excerpt_llama
  confidence_llama
  reasoning_llama
  case_law_excerpt_openai
  legislation_excerpt_openai
  confidence_openai
  reasoning_openai
  paragraphs
  section_text

Distribution by category:
category
a_all_high      7522
b_2high_1low    4192
c_1high_2low    2086
d_all_low        924
Name: count, dtype: int64

First 3 rows (showing key columns):
                para_id                          section_id      category  \
0  eat_2022_192#para_12         id/ukpga/2010/15_section-60    a_all_high   
1  eat_2022_192#para_18  id/ukpga/2010/15_schedule-8-part-1    a_all_high   
2  eat_2022_192#para_21  id/ukpga/2010/15_schedule-1-part-1  b_2high_1low   

  confide

In [86]:
df_final_wide.columns

Index(['para_id', 'section_id', 'category', 'case_law_excerpt_deepseek',
       'legislation_excerpt_deepseek', 'confidence_deepseek',
       'reasoning_deepseek', 'case_law_excerpt_llama',
       'legislation_excerpt_llama', 'confidence_llama', 'reasoning_llama',
       'case_law_excerpt_openai', 'legislation_excerpt_openai',
       'confidence_openai', 'reasoning_openai', 'paragraphs', 'section_text'],
      dtype='object')

In [88]:
df_final_wide.category.unique()

array(['a_all_high', 'b_2high_1low', 'c_1high_2low', 'd_all_low'],
      dtype=object)

In [102]:
print(df_final_wide[df_final_wide['category'] == 'a_all_high']['para_id'].nunique())
print(df_final_wide[df_final_wide['category'] == 'b_2high_1low']['para_id'].nunique())

print(df_final_wide[df_final_wide['category'] == 'c_1high_2low']['para_id'].nunique())
print(df_final_wide[df_final_wide['category'] == 'd_all_low']['para_id'].nunique())




4631
2604
1229
538


In [103]:
df_final_wide.to_csv('../data/final_test/final/reexperiment/fewhot/11August/df_final_wide_analysis.csv', index=False)

In [ ]:
df_source_all_low.to_csv('../data/final_test/final/reexperiment/fewhot/11August/df_source_all_low_for_claude.csv', index=False)

In [3]:
claude_regenrated = pd.read_csv('../data/final_test/final/reexperiment/fewhot/11August/df_source_all_low_for_claude_output.csv')


In [4]:
print(claude_regenrated[claude_regenrated['confidence'] == 'High']['para_id'].nunique())


27


In [5]:
df_final_wide = pd.read_csv('../data/final_test/final/reexperiment/fewhot/11August/df_final_wide_analysis.csv')

In [9]:
set(claude_regenrated[claude_regenrated['confidence'] == 'High']['para_id'].unique()) - set(df_final_wide['para_id'].unique())

set()

In [10]:
#select the df_final_wide with category 'd_all_low' to compare with Claude's output
df_final_wide_d_all_low = df_final_wide[df_final_wide['category'] == 'd_all_low']

In [13]:
regenerated_Success = set(claude_regenrated[claude_regenrated['confidence'] == 'High']['para_id'].unique())

In [14]:
for_review_para_ids = set(df_final_wide_d_all_low.para_id.unique()) - regenerated_Success 

In [15]:
len(for_review_para_ids)

511

In [16]:
df_source_for_review = df_source[df_source['para_id'].isin(for_review_para_ids)]

In [18]:
df_source_for_review.to_csv('../data/final_test/final/reexperiment/fewhot/11August/df_source_for_review.csv', index=False)